In [35]:
from collections import deque
q = deque({1,2,3})
(q,q)
[{11,2},{2,3}]
{1:{1,2}}
for i in {(1,2),(3,4)}:
    print(i)

(1, 2)
(3, 4)


In [63]:
from typing import List
from collections import deque
from pprint import pprint


class Solution:
    def containVirus(self, isInfected: List[List[int]]) -> int:
        rows, cols = len(isInfected), len(isInfected[0])
        ans = 0
        while True:
            # 1.找各个病毒区域并做id标记
            def bfsFind(r, c):
                q = deque([(r, c)])
                vis = {(r, c)}
                while q:
                    r, c = q.popleft()
                    for dx, dy in (1, 0), (-1, 0), (0, 1), (0, -1):
                        if 0 <= (nr := r + dx) < rows and 0 <= (nc := c + dy) < cols and isInfected[nr][nc] == 1 and (nr, nc) not in vis:
                            vis.add((nr, nc))
                            q.append((nr, nc))
                return vis

            virus = {}  # id-当前病毒区域
            allVis = set()  # 防止重复遍历
            id = 0
            for r in range(rows):
                for c in range(cols):
                    if isInfected[r][c] == 1 and (r, c) not in allVis:
                        tmp = bfsFind(r, c)
                        virus[id] = tmp
                        allVis |= tmp
                        id += 1
            if not virus:
                break
            # print(virus)

            # 2.找最危险的病毒区域id(能污染最多的)
            def findDanger(virus):
                vis = set()
                cnt = 0
                for r, c in virus:
                    for dx, dy in (1, 0), (-1, 0), (0, 1), (0, -1):
                        if 0 <= (nr := r + dx) < rows and 0 <= (nc := c + dy) < cols and isInfected[nr][nc] == 0 and (nr, nc) not in vis:
                            vis.add((nr, nc))
                            cnt += 1
                return cnt, vis

            dangerId = -1
            cnt = 0
            border = {}
            for i, v in virus.items():
                tmp_cnt, tmp_vis = findDanger(v)
                border[i] = tmp_vis
                if tmp_cnt > cnt:
                    cnt = tmp_cnt
                    dangerId = i
            if dangerId == -1:
                break
            
            # 3.给危险区域计算所需的挡板
            for r, c in border[dangerId]:
                for dx, dy in (1, 0), (-1, 0), (0, 1), (0, -1):
                    if 0 <= (nr := r + dx) < rows and 0 <= (nc := c + dy) < cols and (nr, nc) in virus[dangerId]:
                        # print(nr,nc,r,c)
                        ans += 1
            # print(ans)
            # 4.给危险区域设置2表示已隔离 其他病毒外扩
            for r, c in virus[dangerId]:
                isInfected[r][c] = 2
            border.pop(dangerId)
            for v in border.values():
                for r, c in v:
                    isInfected[r][c] = 1
            # pprint(isInfected)

        return ans


s = Solution()
# s.containVirus([[0, 1, 0, 0, 0, 0, 0, 1], [0, 1, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, 0, 0]])
# s.containVirus([[1, 1, 1, 0, 0, 0, 0, 0, 0], [1, 0, 1, 0, 1, 1, 1, 1, 1], [1, 1, 1, 0, 0, 0, 0, 0, 0]])
# s.containVirus([[1,1,1],[1,0,1],[1,1,1]])
s.containVirus([[1]])

0

In [64]:
from typing import List
from collections import deque
from pprint import pprint


class Solution:
    def containVirus(self, isInfected: List[List[int]]) -> int:
        rows, cols = len(isInfected), len(isInfected[0])
        ans = 0
        while True:
            # 1.找各个病毒区域并做id标记
            def bfsFind(r, c):
                q = deque([(r, c)])
                vis = {(r, c)}
                while q:
                    r, c = q.popleft()
                    for dx, dy in (1, 0), (-1, 0), (0, 1), (0, -1):
                        if 0 <= (nr := r + dx) < rows and 0 <= (nc := c + dy) < cols and isInfected[nr][nc] == 1 and (nr, nc) not in vis:
                            vis.add((nr, nc))
                            q.append((nr, nc))
                return vis

            virus = {}  # id-当前病毒区域
            allVis = set()  # 防止重复遍历
            id = 0
            for r in range(rows):
                for c in range(cols):
                    if isInfected[r][c] == 1 and (r, c) not in allVis:
                        tmp = bfsFind(r, c)
                        virus[id] = tmp
                        allVis |= tmp
                        id += 1
            if not virus:
                break
            # print(virus)

            # 2.找最危险的病毒区域id(能污染最多的)及所需挡板
            def findDanger(virus):
                vis = set()
                cnt = 0 # 会感染的块
                baffle = 0 # 挡板数量  对于上一个代码改变了计算挡板数量
                for r, c in virus:
                    for dx, dy in (1, 0), (-1, 0), (0, 1), (0, -1):
                        if 0 <= (nr := r + dx) < rows and 0 <= (nc := c + dy) < cols and isInfected[nr][nc] == 0:
                            baffle+=1
                            if (nr, nc) not in vis:
                                vis.add((nr, nc))
                                cnt += 1
                            
                return cnt, vis,baffle

            dangerId = -1
            cnt = 0
            border = {}
            baffle = {}
            for i, v in virus.items():
                tmp_cnt, tmp_vis,tmp_baffle = findDanger(v)
                border[i] = tmp_vis
                baffle[i] = tmp_baffle
                if tmp_cnt > cnt:
                    cnt = tmp_cnt
                    dangerId = i
            if dangerId == -1:
                break
            
            # 3.给危险区域累计所需的挡板
            ans += baffle[dangerId]
            
            # print(ans)
            # 4.给危险区域设置2表示已隔离 其他病毒外扩
            for r, c in virus[dangerId]:
                isInfected[r][c] = 2
            border.pop(dangerId)
            for v in border.values():
                for r, c in v:
                    isInfected[r][c] = 1
            # pprint(isInfected)

        return ans


s = Solution()
# s.containVirus([[0, 1, 0, 0, 0, 0, 0, 1], [0, 1, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, 0, 0]])
# s.containVirus([[1, 1, 1, 0, 0, 0, 0, 0, 0], [1, 0, 1, 0, 1, 1, 1, 1, 1], [1, 1, 1, 0, 0, 0, 0, 0, 0]])
# s.containVirus([[1,1,1],[1,0,1],[1,1,1]])
s.containVirus([[1]])

0